In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('fede'))
if module_path not in sys.path:
    sys.path.append(module_path)
from client import Client
import pandas as pd
from supported_modles import Supported_modles
import utils
from fedavg import Fedavg
from sklearn.utils.class_weight import compute_sample_weight
import numpy as np
from sklearn.metrics import f1_score
import random
from copy import deepcopy

In [2]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [4]:
clients, test_x, test_y = utils.set_data(False, False)

Creating node1.
Creating node2.
Creating node3.
Creating node4.
Creating node5.


In [11]:
selected_model = Supported_modles.MLP_classifier

In [12]:
f1_before = []
for client in clients:
    client.init_empty_model(selected_model,0.01)
    client.train_model()
    # score = client.test_model_f1()
    y_hat = client.model.predict(test_x)
    score = f1_score(test_y,y_hat, average="binary")
    f1_before.append(score)
    print(score)

0.4360285063990149
0.49486969944610915


In [13]:
fedavg = Fedavg("global", 0.1)

[Errno 98] Address already in use
Waitiing for a Connection..


In [14]:
for client in clients:
    client.split_data(10)

In [15]:
###First learn model on clients:
round_weights = []
dataset_size = 0
applicable_models = []
first = True

for client in clients:
    client.init_empty_model(selected_model,0.01)
    
    X_train = client.x_chunks[0]
    y_train = client.y_chunks[0]

    client.round_number = 1

    client.train_model(X_train,y_train)
    dataset_size += client.x.shape[0]
    round_weights.append(client.x.shape[0])
    applicable_models.append(client.model)
    if first:
        fedavg.model = client.model
        first = False
    print('.')

round_weights = np.array(round_weights) / dataset_size # calculate weight based on actual dataset size
fedavg.update_global_model(applicable_models, round_weights, selected_model)

.
.


/home/tk/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:377: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a = np.asanyarray(a)


In [16]:
y_hat = fedavg.model.predict(test_x)
score = f1_score(test_y,y_hat)
print(score)

0.45586513697026454


In [20]:
number_of_rounds = 9
epochs = 200
max_score = 0
optimal_model = None

for round in range(number_of_rounds):
    print(round + 1, end=' ')

    applicable_clients = random.sample((clients),2)# random.randint(3,3))
    applicable_models = []
    applicable_name = []
    round_weights = []
    dataset_size = 0

    for client in applicable_clients:
        # print(f'Client name: {client.name}')

        # X_train, X_test, y_train, y_test = client.split_data()        

        X_train = client.x_chunks[round + 1]
        y_train = client.y_chunks[round + 1]

        ##
        ## Try to use all the data
        ##
        # client.round_number += 1

        # if client.round_number == 10:
        #     clients.remove(client)
        #     print(f'client {client.name} removed.')
       
        dataset_size += X_train.shape[0]
        sample_weights = compute_sample_weight('balanced', y=y_train)

        fedavg.load_global_model(client.model, selected_model)
        try:
            fedavg.train_local_agent(X_train, y_train, client.model, epochs, sample_weights, selected_model)
        except ValueError:
            continue
        round_weights.append(X_train.shape[0])
        applicable_models.append(client.model)


    round_weights = np.array(round_weights) / dataset_size # calculate weight based on actual dataset size
    try:
        fedavg.update_global_model(applicable_models, round_weights, selected_model)
    except ZeroDivisionError:
        continue

    
    y_hat = fedavg.model.predict(test_x)
    score = f1_score(test_y,y_hat)
    if score > max_score:
        print(score)
        max_score = score
        optimal_model = deepcopy(fedavg.model)

1 

/home/tk/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:377: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a = np.asanyarray(a)


0.08608957113467322
2 3 

/home/tk/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:377: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a = np.asanyarray(a)


ZeroDivisionError: Weights sum to zero, can't be normalized

In [11]:
x = test_x
y_hat = fedavg.model.predict(x)
score = f1_score(test_y,y_hat)
print(score)
y_hat = optimal_model.predict(x)
score = f1_score(test_y,y_hat)
print(score)

0.664110926741813
0.6641364052487031


In [12]:
i = 0
for client in clients:
    print(f'Difference {score-f1_before[i]}')
    i += 1

Difference -0.046764424611234534
Difference -0.0487887138033799
Difference -0.04696718692408208
Difference -0.04825297528227046
Difference -0.051224557570334506


## Centralized Zone!

In [ ]:
client1 = Client("node1","0.0.0.0", 5001)

Creating node1.


In [ ]:
dataset = client1.load_data('datasets/Wednesday-workingHours.pcap_ISCX.csv', True)

client1.preprocess_data(dataset, True)

client1.downsample_data(['Destination Port', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets','Total Length of Fwd Packets'])

client1.prep_data()

KeyboardInterrupt: 

In [ ]:
X = client1.x
y = client1.y

client1.x = X[:200000]
client1.y = y[:200000]

test_x = X[600000:]
test_y = y[600000:]

In [ ]:
client1.init_empty_model(selected_model,0.01)
client1.train_model()

In [ ]:
y_hat = client1.model.predict(test_x)
score = f1_score(test_y,y_hat, average="binary")
print(score)

0.9661761376499887
